In [111]:
import requests
import re
from bs4 import BeautifulSoup
import os
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import f1_score, recall_score,precision_score,accuracy_score,roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB

In [26]:

def sep_words(p):
    """this function seperate a sentence string to word strings"""
    words = []
    index = 0 #go through all the letters
    start = 0 #markdown the start position of each word  
    while index < len(p):
        start = index
        while p[index] != " " and p[index] not in[".", ","]:
            index += 1   
            if index == len(p):  
                break   
        words.append(p[start:index])
        if index == len(p):
            break
        while p[index] == " " or p[index] in [".", ","]:
            index += 1
            if index ==len(p):
                break
    return words
            

def savesong(artist,song,content):
    f=open(f'{song}.text','w')
    f.write(str(content))
    f.close()
    


In [27]:
def lownloadsongs(artist,url):
    url=url
    response=requests.get(url)
    links_nw=re.findall('<a href="\/lyric\/.+?>',response.text)
    nw_soup=BeautifulSoup(response.text)
    list1=nw_soup.body.find_all('a')
    link2=[]
    for link in list1:
        link2.append(link.get('href'))
    song=[]
    for link in link2:
        if link.startswith('/lyric/'):
            song.append(link)

#     get the songs url
    songurl=[]
    pre='https://www.lyrics.com/'
    for item in song:
        url=pre+item
        songurl.append(url)
        
 
    if not os.path.exists(artist):
        # check whether this dictionary exist
        os.makedirs(artist)
      
# get lyrics
    for i in tqdm(songurl):
        lyric_response=requests.get(i)
        lyric_soup=BeautifulSoup(lyric_response.text)
        try:
            data=lyric_soup.body.find('pre',class_="lyric-body").text.replace('\n', ' ')
            #x=sep_words(data)
            p=songurl.index(i)
            file_path= f'./{artist}/{p}.text'
            file_path_dir = os.path.dirname(file_path)      #get the file dictionary
           
            f=open(file_path,'w')
            f.write(str(data))
            f.close
        except:
            print(i)

        

In [4]:

# lownloadsongs('nightwish_new','https://www.lyrics.com/artist/Nightwish/373181')



In [5]:

#lownloadsongs('Ozzy-Osbourne','https://www.lyrics.com/artist/Ozzy-Osbourne/5079')

In [28]:
#  read data from files
def mak_corpus(artist,filepath):
    a=os.listdir(filepath)
    bigtext=[]
    art=[]
    for i in a:
        f = open(filepath+i, 'r') 
        c = f.read()
        bigtext.append(c)
        art.append(artist)
        f.close()
    return bigtext,art

In [7]:
filepath_nw='C:\\Users\\zhang\\spiced_bootcamp\\downloaded_materials\\nightwish_new\\'

In [8]:
filepath_oz='C:\\Users\\zhang\\spiced_bootcamp\\downloaded_materials\\Ozzy-Osbourne\\'

In [51]:
niwi= mak_corpus('nightwish',filepath_nw)
ozzy=mak_corpus('ozzy',filepath_oz)
# niwi

In [30]:
# take the corpus and label from the last step
corpus_nw=niwi[0]
label_nw=niwi[1]
corpus_oz=ozzy[0]
label_oz=ozzy[1]

# change corpus and label to Dataframe and merge them

dnw={}
dnw['lyric']=niwi[0]
dnw['artist']=niwi[1]
dfnw=pd.DataFrame(dnw)

doz={}
doz['lyric']=ozzy[0]
doz['artist']=ozzy[1]
dfoz=pd.DataFrame(doz)



In [52]:
df=pd.concat([dfnw,dfoz])
df=df.reset_index(drop=True)
# df

In [54]:
df_shuffled = shuffle(df)
df_shuffled
X=df_shuffled['lyric']
y=df_shuffled['artist']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
# X_train
# y_train

In [33]:
X_train_lst=X_train.tolist()
X_test_lst=X_test.tolist()
y_train_lst=y_train.tolist()
y_test_lst=y_test.tolist()

In [34]:
len(corpus_nw)==len(label_nw)


True

In [64]:
vectorizer=CountVectorizer(stop_words='english')
Xtrain=vectorizer.fit_transform(X_train_lst)

X_y=pd.DataFrame(Xtrain.todense(),columns=vectorizer.get_feature_names(),index=y_train_lst)
y=X_y.index.tolist()



In [150]:
Xtest=vectorizer.transform(X_test_lst)
Xtest_=pd.DataFrame(Xtest.todense(),columns=vectorizer.get_feature_names(),index=y_test_lst)
ytest=Xtest_.index.tolist()

In [161]:
# X_y.reset_index(inplace=True)
# 

In [153]:
X_y.drop(['index','level_0'],axis=1,inplace=True)


In [154]:
X_y

,1000,1001,13th,1st,2005,25,30,3rd,7th,abandon,...,yuletide,zero,zillion,zodiac,zombie,zone,zoo,zoom,zoos,élan
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1278,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1279,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1280,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1281,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [155]:
lr_classifier=LogisticRegression().fit(X_y,y)

In [156]:
X_y

,1000,1001,13th,1st,2005,25,30,3rd,7th,abandon,...,yuletide,zero,zillion,zodiac,zombie,zone,zoo,zoom,zoos,élan
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1278,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1279,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1280,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1281,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [160]:
import operator
print(operator.itemgetter(*np.argsort(lr_classifier.coef_[0]))(vectorizer.get_feature_names())[-20:])
print(operator.itemgetter(*np.argsort(lr_classifier.coef_[0]))(vectorizer.get_feature_names())[:20])

('steal', 'mind', 'got', 'bless', 'thought', 'think', 'oh', 'god', 'll', 'head', 'ha', 'won', 'don', 'just', 'stood', 'stand', 'years', 'know', 've', 'yeah')
('heart', 'instrumental', 'beauty', 'sea', 'heaven', 'did', 'ones', 'night', 'ocean', 'tale', 'beautiful', 'blue', 'far', 'wish', 'earth', 'child', 'home', 'rest', 'lost', 'dream')


In [141]:
Xtest_.index.value_counts()

nightwish    335
ozzy         298
dtype: int64

In [157]:
y_test_pre=lr_classifier.predict(Xtest)
y_train_pre=lr_classifier.predict(X_y)

In [158]:
# f1_score, recall_score,precision_score,accuracy_score
# This happens because for precision/recall are defined using true positives, 
# so it wants to know which of your two classes should be considered "positive".


print('LR accuracy score on train data is', accuracy_score(y, y_train_pre))
print('LR recall score on train data is', recall_score(y, y_train_pre,pos_label='nightwish'))
print('LR precision score on train data is' ,precision_score(y, y_train_pre,pos_label='nightwish'))
print('LR f1 score on train data is' ,f1_score(y, y_train_pre,pos_label='nightwish'))
# print('roc_auc_score score on train data is' ,roc_auc_score(y, y_train_pre))
print('---------------------------')
print('LR accuracy score on test data is', accuracy_score(ytest, y_test_pre))
print('LR recall score on test data is', recall_score(ytest, y_test_pre,pos_label='nightwish'))
print('LR precision score on test data is' ,precision_score(ytest, y_test_pre,pos_label='nightwish'))
print('LR f1 score on test data is' ,f1_score(ytest, y_test_pre,pos_label='nightwish'))



LR accuracy score on train data is 0.9992205767731879
LR recall score on train data is 1.0
LR precision score on train data is 0.9984326018808778
LR f1 score on train data is 0.9992156862745099
---------------------------
LR accuracy score on test data is 1.0
LR recall score on test data is 1.0
LR precision score on test data is 1.0
LR f1 score on test data is 1.0


In [83]:
lr_classifier.score(X_y,y)

0.9992205767731879

In [50]:
lr_classifier.score(Xtest,ytest)

1.0

In [118]:
u= (["On his way to dinner when it took him by surprise And with one pull of the trigger he would vanish overnight Dancing by the roadside, holding on for dear life Then a gun from out of nowhere made a widow of his wife  I don't mind, single file down the runway Feelin' fine and I see you, my friend Over and over again  Who can we get on the case? We need Perry Mason Someone to put you in place Calling Perry Mason again Again  Riding painted horses, oh the kids they love it so You can see it on their faces, how they love the wind to blow Minding my own business like my mama always said But if I don't try to help 'em they could wind up on the front page  I don't mind, draw the line, then draw me an arrow Feelin' fine, then I'll see you, my friend Over and over again  Who can we get on the case? We need Perry Mason Someone to put you in place Calling Perry Mason again Again  Wake me when it's over, tell me it's all right Just keep on talking, baby, I've been doing this all night How much did you give me, tell me it'll be all right  Then I'll see you, my friend Over and over again  Who can we get on the case? We need Perry Mason Someone to put you in place Calling Perry Mason again Again, again, again"])

In [119]:
u1=vectorizer.transform(u)

In [120]:
lr_classifier.predict(u1)

array(['ozzy'], dtype='<U9')

In [121]:
mnb.predict(u1)

array(['ozzy'], dtype='<U9')

In [84]:
clf = RandomForestClassifier(max_depth=4, random_state=0)

In [85]:
clf.fit(X_y,y)


RandomForestClassifier(max_depth=4, random_state=0)

In [100]:
rfy_test_pre=clf.predict(Xtest)

In [99]:
rfy_train_pre=clf.predict(X_y)

In [163]:
print('RF accuracy score on train data is', accuracy_score(y, rfy_train_pre))
print('RF recall score on train data is', recall_score(y, rfy_train_pre,pos_label='nightwish'))
print('RF precision score on train data is' ,precision_score(y, rfy_train_pre,pos_label='nightwish'))
print('RF f1 score on train data is' ,f1_score(y, rfy_train_pre,pos_label='nightwish'))
# print('roc_auc_score score on train data is' ,roc_auc_score(y, y_train_pre))
print('---------------------------')
print('RF accuracy score on test data is', accuracy_score(ytest, rfy_test_pre))
print('RF recall score on test data is', recall_score(ytest, rfy_test_pre,pos_label='nightwish'))
print('RF precision score on test data is' ,precision_score(ytest, rfy_test_pre,pos_label='nightwish'))
print('RF f1 score on test data is' ,f1_score(ytest, rfy_test_pre,pos_label='nightwish'))


RF accuracy score on train data is 0.9883086515978177
RF recall score on train data is 0.9952904238618524
RF precision score on train data is 0.9814241486068112
RF f1 score on train data is 0.9883086515978177
---------------------------
RF accuracy score on test data is 0.990521327014218
RF recall score on test data is 0.991044776119403
RF precision score on test data is 0.991044776119403
RF f1 score on test data is 0.991044776119403


In [132]:
gnb = GaussianNB()
gnb.fit(X_y,y)
gnby_test_pre=gnb.predict(Xtest.toarray())
gnby_train_pre=gnb.predict(X_y)

In [165]:
print('gnb accuracy score on train data is', accuracy_score(y, gnby_train_pre))
print('gnb recall score on train data is', recall_score(y, gnby_train_pre,pos_label='nightwish'))
print('gnb precision score on train data is' ,precision_score(y, gnby_train_pre,pos_label='nightwish'))
print('gnb f1 score on train data is' ,f1_score(y, gnby_train_pre,pos_label='nightwish'))
# print('roc_auc_score score on train data is' ,roc_auc_score(y, y_train_pre))
print('---------------------------')
print('gnb accuracy score on test data is', accuracy_score(ytest, gnby_test_pre))
print('gnb recall score on test data is', recall_score(ytest, gnby_test_pre,pos_label='nightwish'))
print('gnb precision score on test data is' ,precision_score(ytest, gnby_test_pre,pos_label='nightwish'))
print('gnb f1 score on test data is' ,f1_score(ytest, gnby_test_pre,pos_label='nightwish'))


gnb accuracy score on train data is 0.9922057677318784
gnb recall score on train data is 0.9843014128728415
gnb precision score on train data is 1.0
gnb f1 score on train data is 0.9920886075949367
---------------------------
gnb accuracy score on test data is 0.985781990521327
gnb recall score on test data is 0.9791044776119403
gnb precision score on test data is 0.9939393939393939
gnb f1 score on test data is 0.9864661654135337


In [114]:
MultinomialNB
mnb = MultinomialNB()
mnb.fit(X_y,y)
mnby_test_pre=gnb.predict(Xtest)
mnby_train_pre=gnb.predict(X_y)

In [164]:
print('mnb accuracy score on train data is', accuracy_score(y, mnby_train_pre))
print('mnb recall score on train data is', recall_score(y, mnby_train_pre,pos_label='nightwish'))
print('mnb precision score on train data is' ,precision_score(y, mnby_train_pre,pos_label='nightwish'))
print('mnb f1 score on train data is' ,f1_score(y, mnby_train_pre,pos_label='nightwish'))
# print('roc_auc_score score on train data is' ,roc_auc_score(y, y_train_pre))
print('---------------------------')
print('mnb accuracy score on test data is', accuracy_score(ytest, mnby_test_pre))
print('mnb recall score on test data is', recall_score(ytest, mnby_test_pre,pos_label='nightwish'))
print('mnb precision score on test data is' ,precision_score(ytest, mnby_test_pre,pos_label='nightwish'))
print('mnb f1 score on test data is' ,f1_score(ytest, mnby_test_pre,pos_label='nightwish'))


mnb accuracy score on train data is 0.990646921278254
mnb recall score on train data is 0.9843014128728415
mnb precision score on train data is 0.9968203497615262
mnb f1 score on train data is 0.990521327014218
---------------------------
mnb accuracy score on test data is 0.9826224328593997
mnb recall score on test data is 0.9791044776119403
mnb precision score on test data is 0.9879518072289156
mnb f1 score on test data is 0.9835082458770614


In [129]:
import argparse

parser = argparse.ArgumentParser()
parser.add_argument('-from', type=str)
parser.add_argument('-to', type=str, default='everyone')

args = parser.parse_args('-from yitian'.split(' '))
print(args)

Namespace(from='yitian', to='everyone')


In [130]:
'-from yitian'.split(' ')

['-from', 'yitian']

https://www.youtube.com/watch?v=GiqjQx-YZd0

https://chairnerd.seatgeek.com/fuzzywuzzy-fuzzy-string-matching-in-python/


https://www.nltk.org/_modules/nltk/stem/porter.html